In [61]:
# Import dependencies
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import os
from datetime import datetime
import requests
import pandas_datareader as pdr
from datetime import datetime
import pandas_datareader as pdr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
import hvplot.pandas
from keras.models import Sequential
from tensorflow.keras.layers import LSTM
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
%matplotlib inline
warnings.filterwarnings('ignore')

In [62]:
# Function to Validate Tickers
def validateTick(tick):
    
    # Try Inputting into Yahoo Stocks
    try:
        pdr.DataReader(f'{tick}','yahoo',date)
    
    # Raise Error if Invalid
    except:
        raise ValueError("Ticker Not Valid.")
        
# Function to Validate Date
def validate(date_text):
    
    # Try Check on Format
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        
# Function to Validate Date
def validateShift(shift):
    shift = int(shift)
    # Try Check on Format
    try:
        if 0 > shift <= 365:
            pass
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Please enter a whole number.")

In [63]:
# Pick a Start Date
date = input("Pick a start date for your analysis (YYYY-MM-DD): ")

# Validate Correct Date Format
validate(date)

Pick a start date for your analysis (YYYY-MM-DD):  2016-01-01


In [64]:
# Pick a Stock
stock_selection = input("Pick a stock ticker for your analysis: ")

# Validate Correct Date Format
validateTick(stock_selection)

Pick a stock ticker for your analysis:  CME


In [65]:
# Pick Shift
shift_selection = input("Pick how many days to shift your close values for your analysis: ")

# Validate Correct Date Format
validateShift(shift_selection)

Pick how many days to shift your close values for your analysis:  1


In [66]:
# Function to Create Stocks
def stockCreation(date, stock_selection, days_shifted):
    
    # Read Stock Data
    globals()[stock_selection] = pdr.DataReader(f'{stock_selection}','yahoo',date)
    
    # Create Shifted Close Column
    globals()[stock_selection]["Shifted Close"] = globals()[stock_selection]["Adj Close"].shift(-int(days_shifted))
    
    # Drop Rows Without Shift Values
    globals()[stock_selection].drop(globals()[stock_selection].tail(int(days_shifted)).index, inplace = True) 
    
    # Empty List to Hold Profit or Loss
    profit_loss = []
    
    # Iterate Through DataFrame
    for x in range(len(globals()[stock_selection])):
        
        # Handle Profitable Days
        if globals()[stock_selection]["Shifted Close"][x] > globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Profit" to Empty List
            profit_loss.append("Profit")
            
        # Handle Loss Days
        elif globals()[stock_selection]["Shifted Close"][x] < globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Loss" to Empty List
            profit_loss.append("Loss")
            
        # Handle Push Days
        elif globals()[stock_selection]["Shifted Close"][x] == globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Push" to Empty List
            profit_loss.append("Push")
    
    # Create Profit/Loss Column
    globals()[stock_selection]["Profit/Loss"] = profit_loss
    
    # Create Returns Column
    globals()[stock_selection]['Returns'] = np.log(globals()[stock_selection]['Adj Close'] / globals()[stock_selection]['Adj Close'].shift(1))
    
    # Iterate to Create STD Columns
    for x in range(5, 20, 5):
        
        # Create STD Column Based on Iteration
        col_name = 'std_' + str(x)
        
        # Add STD Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).std()

    # Iterate to Create MA Columns
    for x in range(10, 30, 5):
        
        # Create MA Column Based on Iteration
        col_name = 'ma_' + str(x)
        
        # Add MA Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).mean()

    # Iterate to Create % Change Columns
    for x in range(3, 12, 3):
        
        # Create % Change Column Based on Iteration
        col_name = 'pct_' + str(x)
        
        # Add % Change Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].pct_change().rolling(window=x).sum()

    # Create VMA 4 Column
    col_name = 'vma_4'
    
    # Add VMA 4 Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Volume'].rolling(4).mean()

    # Create Intraday Column
    col_name = 'co'
    
    # Add Intraday Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'] - globals()[stock_selection]['Open']
    
    # Drop All NaN's
    globals()[stock_selection].dropna(inplace=True)
    
    # Return DataFrame
    return globals()[stock_selection]

In [67]:
# Create Stock
stockdf = stockCreation(date, stock_selection, shift_selection)
stockdf.head()

,High,Low,Open,Close,Volume,Adj Close,Shifted Close,Profit/Loss,Returns,std_5,...,std_15,ma_10,ma_15,ma_20,ma_25,pct_3,pct_6,pct_9,vma_4,co
Date,,,,,,,,,,,,,,,,,,,,,
2016-02-08,90.669998,87.000000,88.410004,90.190002,2758100.0,75.330887,77.936836,Profit,0.018351,1.218740,...,2.022289,73.756442,72.653080,72.354897,72.608648,0.009896,0.004898,0.044908,2910525.0,-13.079117
2016-02-09,93.809998,89.050003,89.050003,93.309998,3233500.0,77.936836,75.614868,Loss,0.034008,2.126921,...,2.423534,74.336102,73.053999,72.620506,72.763668,0.078829,0.044054,0.075449,3200800.0,-11.113167
2016-02-10,93.489998,90.440002,93.029999,90.529999,2133700.0,75.614868,74.604225,Loss,-0.030246,2.154174,...,2.382844,74.654331,73.406474,72.757486,72.802423,0.023321,0.017951,0.041389,2785025.0,-17.415131
2016-02-11,90.550003,88.320000,88.360001,89.320000,1952800.0,74.604225,76.132721,Profit,-0.013456,1.512457,...,1.967143,74.840592,73.814632,72.972562,72.815453,-0.008565,0.001331,-0.003668,2519525.0,-13.755775
2016-02-12,91.290001,89.510002,90.919998,91.150002,1691200.0,76.132721,75.765213,Loss,0.020281,1.253418,...,1.863000,74.949174,74.169889,73.241928,72.936396,-0.022671,0.056159,0.021383,2252800.0,-14.787277


In [68]:
stockdf.info

<bound method DataFrame.info of                   High         Low        Open       Close     Volume  \
Date                                                                    
2016-02-08   90.669998   87.000000   88.410004   90.190002  2758100.0   
2016-02-09   93.809998   89.050003   89.050003   93.309998  3233500.0   
2016-02-10   93.489998   90.440002   93.029999   90.529999  2133700.0   
2016-02-11   90.550003   88.320000   88.360001   89.320000  1952800.0   
2016-02-12   91.290001   89.510002   90.919998   91.150002  1691200.0   
...                ...         ...         ...         ...        ...   
2021-02-12  184.589996  182.110001  182.860001  184.369995  1441400.0   
2021-02-16  189.899994  183.000000  185.089996  189.410004  2677700.0   
2021-02-17  191.899994  187.220001  189.419998  191.139999  1513800.0   
2021-02-18  194.240005  189.720001  190.050003  193.990005  1497200.0   
2021-02-19  196.460007  192.889999  194.970001  195.070007  1419300.0   

             Adj C

In [69]:
# Create function for column numbering including features (X) and target (y)
def window_data(df, window, feature_column, target_column):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_column]
        target = df.iloc[(i + window), target_column]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [70]:
# # change profit/loss to 0's/1's
# ## convert from string to integer
profit_loss = []
for row in stockdf.iloc[:,7]:
    if row == 'Loss':
         profit_loss.append(0)
    else:
         profit_loss.append(1)
            
stockdf ["Profit/Loss"] = profit_loss

In [71]:
# Predict Closing Prices
window_size = 100

feature_column = 7
target_column = 1
X, y = window_data(stockdf, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[1 0 0 1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 1
  1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1
  1 1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 1 1]
 [0 0 1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 1 1
  0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1
  1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 1 1 0]
 [0 1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 1 1 0
  0 0 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1
  0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 1 1 0 1]
 [1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0
  0 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0
  0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 1 1 0 1 1]
 [0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0
  0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 0
  1 0 1

In [72]:
# Use 70% of the data for training and the remainder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

In [73]:
# Use the MinMaxScaler to scale data between 0 and 1.
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)
X_train[:5]

array([[1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1.,
        1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
        1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
        0., 1., 1., 1.],
       [0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
        1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
        0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
        1., 1., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
        0., 1.

In [74]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]} \n")

X_train sample values:
[[[1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [1.]
  [1.]
  [0.]
  [1.]
  [1.]
  [1.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [1.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [1.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [0.]
  [1.]
  [1.]
  [1.]]

 [[0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [1.]
  [1.]
  [0.]
  [1.]
  [1.]
  [1.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  

In [75]:
# Build the LSTM model. 

model = Sequential()

number_units = 50
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1,activation='sigmoid'))

In [76]:
# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy")

In [77]:
# Summarize the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 100, 50)           10400     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100, 50)           20200     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_8 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [78]:
# Train the model
model.fit(X_train, y_train, epochs=50, shuffle=False, batch_size=5, verbose=1)

Epoch 1/50
164/164 [==============================] - 14s 58ms/step - loss: 0.5067
Epoch 2/50
164/164 [==============================] - 9s 57ms/step - loss: 0.8557
Epoch 3/50
164/164 [==============================] - 10s 60ms/step - loss: 0.9225
Epoch 4/50
164/164 [==============================] - 10s 61ms/step - loss: 0.7360
Epoch 5/50
164/164 [==============================] - 10s 60ms/step - loss: 0.6914
Epoch 6/50
164/164 [==============================] - 10s 62ms/step - loss: 0.6780
Epoch 7/50
164/164 [==============================] - 10s 60ms/step - loss: 0.6702
Epoch 8/50
164/164 [==============================] - 12s 71ms/step - loss: 0.6662
Epoch 9/50
164/164 [==============================] - 11s 65ms/step - loss: 0.6644
Epoch 10/50
164/164 [==============================] - 11s 69ms/step - loss: 0.6618
Epoch 11/50
164/164 [==============================] - 19s 119ms/step - loss: 0.6612
Epoch 12/50
164/164 [==============================] - 9s 58ms/step - loss: 0.6588
Ep

In [79]:
# Evaluate the model
model.evaluate(X_test, y_test)

11/11 [==============================] - 1s 19ms/step - loss: 0.7415


0.74153733253479

In [80]:
# Predictions
predicted = model.predict(X_test)

In [81]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [82]:
# Comparing Real and Predicted values
stockd = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = stockdf.index[-len(real_prices): ]) 
stockd.head()

,Real,Predicted
Date,,
2019-09-30,210.589996,151.528366
2019-10-01,208.320007,151.528366
2019-10-02,206.880005,151.528366
2019-10-03,207.699997,151.528366
2019-10-04,209.960007,151.528366


In [83]:
# Plot the real vs predicted values as a line chart
stockd.hvplot(title="Real vs Predicted Stock Price", xlabel="Real", ylabel="Predicted")

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [87]:
# Printing metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred)

0.08208880658954279

In [88]:
print(f"LSTM is {rms}.")

LSTM is 0.2865114423361531.
